In [1]:
!pip install python-box timm

In [2]:
import os
import warnings
from pprint import pprint
from glob import glob
import numpy as np
import pandas as pd
from box import Box
from timm import create_model
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

import torch
import torch.optim as optim
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.io import read_image
import pytorch_lightning as pl
from pytorch_lightning import callbacks
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

warnings.filterwarnings('ignore')

## Config

In [17]:
config = {
    'seed': 2021,
    'root': '../petfinder-pawpularity-score',
    'n_splits': 5,
    'epochs': 20,
    'image_size': 224,
    'train_loader': {
        'batch_size': 64,
        'shuffle': True,
        'num_workers': 4,
        'pin_memory': False,
        'drop_last': True,
    },
    'val_loader': {
        'batch_size': 64,
        'shuffle': False,
        'num_workers': 4,
        'pin_memory': False,
        'drop_last': False,
    },
    'trainer': {
        'gpus': 1,
        'accelerator': "dp", # when use multiple gpus
        'accumulate_grad_batches': 1,
        'progress_bar_refresh_rate': 1,
        'fast_dev_run': False,
        'num_sanity_val_steps': 0,
        'resume_from_checkpoint': None,
    },
    'model': {
        # EfficientNet
        # 'name': 'tf_efficientnet_b6_ns',
        # 'name': 'tf_efficientnet_b7_ns',

        # EfficientNetv2
        # 'name': 'tf_efficientnetv2_m_in21k',
        # 'name': 'tf_efficientnetv2_l_in21k',
        
        # Swin-Transformer
        'name': 'swin_large_patch4_window7_224_in22k',
        # 'name': 'swin_large_patch4_window12_384_in22k',

        # neck - A: BCELoss, B: ArcFace
        # 'neck': 'option-A',
        'neck': 'option-B',

        'embedding_size': 512,
        'n_classes': 100,
        'output_dim': 1,
    },
    'optimizer': {
        'name': 'optim.AdamW',
        'params': {
            'lr': 1e-5,
        },
    },
    'scheduler': {
        'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
        'params': {
            'T_0': 20,
            'eta_min': 1e-4,
        }
    },
    
    # 'loss': 'nn.BCEWithLogitsLoss',
    'loss': 'ArcFaceLoss',
}

config = Box(config)

## Dataset

In [18]:
class PetfinderDataset(Dataset):
    def __init__(self, df, image_size=224):
        self._X = df["Id"].values
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self._transform = T.Resize([image_size, image_size])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, label
        return image

class PetfinderDataModule(pl.LightningDataModule):
    def __init__(self, train_df, val_df, cfg):
        super().__init__()
        self._train_df = train_df
        self._val_df = val_df
        self._cfg = cfg

    def __create_dataset(self, train=True):
        return (
            PetfinderDataset(self._train_df, self._cfg.image_size)
            if train
            else PetfinderDataset(self._val_df, self._cfg.image_size)
        )

    def train_dataloader(self):
        dataset = self.__create_dataset(True)
        return DataLoader(dataset, **self._cfg.train_loader)

    def val_dataloader(self):
        dataset = self.__create_dataset(False)
        return DataLoader(dataset, **self._cfg.val_loader)

In [19]:
torch.autograd.set_detect_anomaly(True)
seed_everything(config.seed)

df = pd.read_csv(os.path.join(config.root, "train.csv"))
df["Id"] = df["Id"].apply(lambda x: os.path.join(config.root, "train", x + ".jpg"))

Global seed set to 2021


In [20]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB

def get_default_transforms():
    transform = {
        "train": T.Compose(
            [
                T.RandomHorizontalFlip(),
                T.RandomVerticalFlip(),
                T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
        "val": T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
    }
    return transform

## ArcFace

In [21]:
import math

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, eps=1e-7):
        super().__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()


class ArcFaceLoss(nn.Module):
    def __init__(self, s=30.0, m=0.1):
        super().__init__()
        self.crit = nn.BCEWithLogitsLoss()
        self.s = s
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, logits, labels):
        logits = logits.float()
        cosine = logits
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        
        output = (labels * phi) + ((1.0 - labels) * cosine)
        output *= self.s
        loss = self.crit(output, labels)
        return loss / 2

In [22]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform(self.weight)

    def forward(self, features):
        cosine = F.linear(F.normalize(features), F.normalize(self.weight))
        return cosine

In [23]:
def mixup(x: torch.Tensor, y: torch.Tensor, alpha: float = 1.0):
    assert alpha > 0, "alpha should be larger than 0"
    assert x.size(0) > 1, "Mixup cannot be applied to a single instance."

    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(x.size()[0])
    mixed_x = lam * x + (1 - lam) * x[rand_index, :]
    target_a, target_b = y, y[rand_index]
    return mixed_x, target_a, target_b, lam

class Model(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self._criterion = eval(self.cfg.loss)()
        self.transform = get_default_transforms()
        self.save_hyperparameters(cfg)

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=True, num_classes=0, in_chans=3
        )
        num_features = self.backbone.num_features
        if self.cfg.model.neck == "option-A":
            self.neck = nn.Sequential(
                nn.Linear(num_features, 384),
                nn.Dropout(0.1),
                nn.Linear(384, 96), 
                nn.Linear(96, self.cfg.model.output_dim)
            )
        else:
            self.neck = nn.Sequential(
                nn.Linear(num_features, self.cfg.model.embedding_size, bias=False),
                nn.BatchNorm1d(self.cfg.model.embedding_size),
                ArcMarginProduct(self.cfg.model.embedding_size, self.cfg.model.output_dim)
            )

    def forward(self, x):
        f = self.backbone(x)
        out = self.neck(f)
        return out

    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'train')
        return {'loss': loss, 'pred': pred, 'labels': labels}
        
    def validation_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'val')
        return {'pred': pred, 'labels': labels}
    
    def __share_step(self, batch, mode):
        images, labels = batch
        labels = labels.float() / 100.0
        images = self.transform[mode](images)
        
        if torch.rand(1)[0] < 0.5 and mode == 'train':
            mix_images, target_a, target_b, lam = mixup(images, labels, alpha=0.5)
            logits = self.forward(mix_images).squeeze(1)
            loss = self._criterion(logits, target_a) * lam + \
                (1 - lam) * self._criterion(logits, target_b)
        else:
            logits = self.forward(images).squeeze(1)
            loss = self._criterion(logits, labels)
        
        if self.cfg.trainer.gpus == 1:
            pred = logits.sigmoid().detach().cpu() * 100.
            labels = labels.detach().cpu() * 100.
        else:
            pred = self.all_gather(logits.sigmoid().detach().cpu() * 100.)
            labels = self.all_gather(labels.detach().cpu() * 100.)
            
        return loss, pred, labels


    def training_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, 'train')


    def validation_epoch_end(self, outputs):
        self.__share_epoch_end(outputs, 'val')  

        
    def __share_epoch_end(self, outputs, mode):
        preds = []
        labels = []
        for out in outputs:
            pred, label = out['pred'], out['labels']
            preds.append(pred)
            labels.append(label)
        preds = torch.cat(preds)
        labels = torch.cat(labels)
        metrics = torch.sqrt(((labels - preds) ** 2).mean())
        self.log(f'{mode}_loss', metrics, sync_dist=True)
    
    def configure_optimizers(self):
        optimizer = eval(self.cfg.optimizer.name)(
            self.parameters(), **self.cfg.optimizer.params
        )
        scheduler = eval(self.cfg.scheduler.name)(
            optimizer,
            **self.cfg.scheduler.params
        )
        return [optimizer], [scheduler]

In [24]:
skf = StratifiedKFold(n_splits=config.n_splits, shuffle=True, random_state=config.seed)

for fold, (train_idx, val_idx) in enumerate(skf.split(df["Id"], df["Pawpularity"])):
    train_df = df.loc[train_idx].reset_index(drop=True)
    val_df = df.loc[val_idx].reset_index(drop=True)
    dm = PetfinderDataModule(train_df, val_df, config)
    model = Model(config)
    earlystopping = EarlyStopping(monitor="val_loss")
    lr_monitor = callbacks.LearningRateMonitor()
    loss_checkpoint = callbacks.ModelCheckpoint(
        filename="best_loss",
        monitor="val_loss",
        save_top_k=1,
        mode="min",
        save_last=False,
    )
    logger = TensorBoardLogger(config.model.name)
    
    trainer = pl.Trainer(
        logger=logger,
        max_epochs=config.epochs,
        callbacks=[lr_monitor, loss_checkpoint, earlystopping],
        **config.trainer,
    )
    trainer.fit(model, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name       | Type            | Params
-----------------------------------------------
0 | backbone   | SwinTransformer | 194 M 
1 | neck       | Sequential      | 787 K 
2 | _criterion | ArcFaceLoss     | 0     
-----------------------------------------------
195 M     Trainable params
0         Non-trainable params
195 M     Total params
783.134   Total estimated model params size (MB)


Epoch 0:   0%|          | 0/619 [00:00<00:00, 901.81it/s]   

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name       | Type            | Params
-----------------------------------------------
0 | backbone   | SwinTransformer | 194 M 
1 | neck       | Sequential      | 787 K 
2 | _criterion | ArcFaceLoss     | 0     
-----------------------------------------------
195 M     Trainable params
0         Non-trainable params
195 M     Total params
783.134   Total estimated model params size (MB)


Epoch 0:   0%|          | 0/619 [00:00<00:00, 961.56it/s]   

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

path = glob(f'./{config.model.name}/default/version_0/events*')[0]
even_acc = EventAccumulator(path, size_guidance={'scalar': 0})
even_acc.Reload()

scalars = {}
for tag in event_acc.Tags()['scalars']:
    event = event_acc.Scalars(tag)
    scalars[tag] = [event.value for event in events]

In [ ]:
import seaborn as sns
sns.set()

plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.plot(range(len(scalars['lr-AdamW'])), scalars['lr-AdamW'])
plt.xlabel('epoch')
plt.ylabel('lr')
plt.title('adamw lr')

plt.subplot(1, 2, 2)
plt.plot(range(len(scalars['train_loss'])), scalars['train_loss'], label='train_loss')
plt.plot(range(len(scalars['val_loss'])), scalars['val_loss'], label='val_loss')
plt.legend()
plt.ylabel('rmse')
plt.xlabel('epoch')
plt.title('train/val rmse')
plt.show()

In [ ]:
print('best_val_loss', min(scalars['val_loss']))